# Taguer un fichier avec TreeTagger Online depuis Python

In [1]:
import mechanicalsoup, codecs

In [2]:
treetaggerURL="http://cental.fltr.ucl.ac.be/treetagger/"
fichierBrut="test.txt"

In [3]:
browser = mechanicalsoup.Browser()

In [4]:
# request TT home page. the result is a requests.Response object http://docs.python-requests.org/en/latest/user/quickstart/#response-content
accueil = browser.get(treetaggerURL)

Dans la réponse *accueil*, il y a un formulaire qui permet d'envoyer le fichier qu'on veut taguer :
- treetagger_form

In [5]:
print accueil.soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<!-- sessionKey= loginguid= -->
<!-- Variables du bandeau PreExecute -->
<!--tmp-fr/cental.html--> <!--tmp-fr/cental.html--> <head><!-- PageID 44005 - published by RedDot 7.5 - 7.5.2.17 - 14445 -->
<title>Treetagger</title>
<!-- Modele de page : page_deux_colonnes ; date_publ: 20090712T2047 -->
<!--domaine_code=cental-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="http://purl.org/DC/elements/1.1/" rel="schema.DC"/>
<meta content="Recherche au Cental" name="DC.Title"/>
<meta content="Isabelle Lecroart" name="DC.Creator"/>
<meta content="Université catholique de Louvain" name="DC.Subject"/>
<meta content="" name="DC.Description"/>
<meta content="Isabelle Lecroart" name="DC.Publisher"/>
<meta content="" name="DC.Contributor"/>
<meta content="9/03/2009" name="DC.Date"/>
<meta content="text" name="DC.Type"/>
<meta content="text/html" name="DC.Format"/>
<meta content="http://www.uclouv

accueil.soup is a BeautifulSoup object http://www.crummy.com/software/BeautifulSoup/bs4/doc/#beautifulsoup  
we grab the treetagger_form in *formulaire*

In [6]:
formulaire = accueil.soup.select("form[name=treetagger_form]")[0]

to use the form, we need to make it a mechanicalsoup.Form  
and set the **file_to_tag** form field to the name of our file

In [7]:
form=mechanicalsoup.Form(formulaire)
form.set('file_to_tag', fichierBrut)

the filled form can now be uploaded with a browser.submit  
the resulting page is put in *upload*

In [8]:
upload=browser.submit(form,url="http://cental.fltr.ucl.ac.be/treetagger/")

the *upload.soup* contains the extension pointing to the resulting file on the site

In [9]:
print upload.soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<!-- sessionKey= loginguid= -->
<!-- Variables du bandeau PreExecute -->
<!--tmp-fr/cental.html--> <!--tmp-fr/cental.html--> <head><!-- PageID 44005 - published by RedDot 7.5 - 7.5.2.17 - 14445 -->
<title>Treetagger</title>
<!-- Modele de page : page_deux_colonnes ; date_publ: 20090712T2047 -->
<!--domaine_code=cental-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="http://purl.org/DC/elements/1.1/" rel="schema.DC"/>
<meta content="Recherche au Cental" name="DC.Title"/>
<meta content="Isabelle Lecroart" name="DC.Creator"/>
<meta content="Université catholique de Louvain" name="DC.Subject"/>
<meta content="" name="DC.Description"/>
<meta content="Isabelle Lecroart" name="DC.Publisher"/>
<meta content="" name="DC.Contributor"/>
<meta content="9/03/2009" name="DC.Date"/>
<meta content="text" name="DC.Type"/>
<meta content="text/html" name="DC.Format"/>
<meta content="http://www.uclouv

In [10]:
for div in upload.soup.findAll('div',attrs={"class":u"texte-base"}):
    for link in div.findAll("a",attrs={"content":"text/plain;charset=UTF-8"}):
        resultURL=treetaggerURL+link.attrs["href"]
        print resultURL

http://cental.fltr.ucl.ac.be/treetagger/tmp/jGG5UD0UKL.txt


In [11]:
result = browser.get(resultURL)

le fichier étiqueté est disponible dans *result.content* (plutôt que *result.soup* puisque c'est du texte brut et pas du HTML)  
**Attention** le résultat est encodé en *utf8* mais stocké dans une *str*, il faut faire un *decode("utf8")* pour l'obtenir en unicode.

In [21]:
resultLines=result.content.decode("utf8")
print resultLines

Nous	PRO:PER	nous
étions	VER:impf	être
à	PRP	à
l'	DET:ART	le
Étude	NOM	étude
,	PUN	,
quand	KON	quand
le	DET:ART	le
Proviseur	NOM	proviseur
entra	VER:simp	entrer
,	PUN	,
suivi	VER:pper	suivre
d'	PRP	de
un	DET:ART	un
nouveau	ADJ	nouveau
habillé	VER:pper	habiller
en	PRP	en
bourgeois	NOM	bourgeois
et	KON	et
d'	PRP	de
un	DET:ART	un
garçon	NOM	garçon
de	PRP	de
classe	NOM	classe
qui	PRO:REL	qui
portait	VER:impf	porter
un	DET:ART	un
grand	ADJ	grand
pupitre	NOM	pupitre
.	SENT	.
Ceux	PRO:DEM	celui
qui	PRO:REL	qui
dormaient	VER:impf	dormir
se	PRO:PER	se
réveillèrent	VER:simp	réveiller
,	PUN	,
et	KON	et
chacun	PRO:IND	chacun
se	PRO:PER	se
leva	VER:simp	lever
comme	ADV	comme
surpris	VER:pper	surprendre
dans	PRP	dans
son	DET:POS	son
travail	NOM	travail
.	SENT	.
Le	DET:ART	le
Proviseur	NOM	proviseur
nous	PRO:PER	nous
fit	VER:simp	faire
signe	NOM	signe
de	PRP	de
nous	PRO:PER	nous
rasseoir	VER:infi	rasseoir
;	PUN	;
puis	ADV	puis
,	PUN	,
se	PRO:PER	se
tournant	VER:ppre	tourner
vers	PRP	vers
le	DET:ART	l

In [23]:
with codecs.open(fichierBrut.replace(".txt","-TT.txt"),"w",encoding="utf8") as outFile:
    outFile.write(resultLines)